In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df3 = pd.read_csv("data/train_preliminary/click_log.csv")
df_tmp = pd.merge(df3, df1, left_on='user_id', right_on='user_id')
df = pd.merge(df_tmp, df2, left_on='creative_id', right_on='creative_id')
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id']
df = df[order]
df.head()

Using TensorFlow backend.


,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,30920,9,567330,1,504423,3,32638
1,320815,15,567330,1,504423,3,32638
2,355089,11,567330,1,504423,3,32638
3,363442,9,567330,1,504423,3,32638
4,370513,14,567330,1,504423,3,32638


In [2]:
df.sort_values(['user_id', 'time'], inplace=True)
df

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
16204330,1,20,877468,1,773445,5,29455
23067470,1,20,209778,1,188507,2,9702
26781033,1,20,821396,1,724607,5,7293
22502290,1,39,1683713,1,1458878,5,14668
9504689,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
28703035,900000,57,2692144,1,2316309,5,14372
15724163,900000,71,3391205,1,2918341,5,22137
7226547,900000,72,3471208,1,2989104,5,7283
9638586,900000,74,3527098,1,3037879,18,14681


In [3]:
df.reset_index(drop=True, inplace=True)
df

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,1,20,877468,1,773445,5,29455
1,1,20,209778,1,188507,2,9702
2,1,20,821396,1,724607,5,7293
3,1,39,1683713,1,1458878,5,14668
4,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
30082766,900000,57,2692144,1,2316309,5,14372
30082767,900000,71,3391205,1,2918341,5,22137
30082768,900000,72,3471208,1,2989104,5,7283
30082769,900000,74,3527098,1,3037879,18,14681


In [4]:
df['creative_id'].max()

4445718

In [5]:
x_tmp = np.array(df['creative_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(4445719, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['creative_id_0', 'creative_id_1', 'creative_id_2', 'creative_id_3', 'creative_id_4'])
X_tmp

(30082800, 1)
300828/300828 [==============================] - 3s 11us/step


,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4
0,-0.020450,0.026225,-0.028412,-0.023482,-0.020356
1,0.042897,0.042010,-0.013918,0.011240,-0.030617
2,0.023067,0.027716,-0.028125,0.003288,-0.022204
3,-0.041668,-0.005339,-0.003069,-0.022583,-0.040599
4,-0.031345,0.033751,-0.030880,-0.028066,0.028547
...,...,...,...,...,...
30082766,-0.004689,-0.026927,-0.036577,-0.037790,-0.037586
30082767,-0.011292,0.046097,-0.048091,0.031339,-0.008574
30082768,-0.028107,0.048297,-0.041825,0.008949,-0.003579
30082769,0.047436,0.001446,-0.010964,0.002186,0.041727


In [6]:
X = df[['user_id', 'time', 'click_times']]

In [7]:
X = pd.concat([X, X_tmp], axis=1)
X.head()

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4
0,1,20,1,-0.020450,0.026225,-0.028412,-0.023482,-0.020356
1,1,20,1,0.042897,0.042010,-0.013918,0.011240,-0.030617
2,1,20,1,0.023067,0.027716,-0.028125,0.003288,-0.022204
3,1,39,1,-0.041668,-0.005339,-0.003069,-0.022583,-0.040599
4,1,40,1,-0.031345,0.033751,-0.030880,-0.028066,0.028547


In [8]:
df['ad_id'].max()

3812200

In [9]:
x_tmp = np.array(df['ad_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(3812201, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['ad_id_0', 'ad_id_1', 'ad_id_2', 'ad_id_3', 'ad_id_4'])
X_tmp

(30082800, 1)
300828/300828 [==============================] - 3s 11us/step


,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4
0,-0.034429,-0.005427,0.026714,0.036977,0.003526
1,-0.042416,-0.020023,0.015293,-0.038636,0.013217
2,-0.014087,-0.047243,-0.046313,-0.011373,0.010814
3,0.011407,0.027688,0.028993,0.011937,0.037679
4,0.040212,0.015020,0.049602,-0.043647,0.025991
...,...,...,...,...,...
30082766,-0.046403,0.031878,0.009838,-0.019066,0.018234
30082767,0.035545,-0.041778,-0.049822,-0.017856,-0.026819
30082768,-0.021256,0.045389,0.038815,0.040389,0.026119
30082769,-0.016992,-0.040091,0.034721,-0.043670,-0.031338


In [10]:
X = pd.concat([X, X_tmp], axis=1)
X.head()

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4
0,1,20,1,-0.020450,0.026225,-0.028412,-0.023482,-0.020356,-0.034429,-0.005427,0.026714,0.036977,0.003526
1,1,20,1,0.042897,0.042010,-0.013918,0.011240,-0.030617,-0.042416,-0.020023,0.015293,-0.038636,0.013217
2,1,20,1,0.023067,0.027716,-0.028125,0.003288,-0.022204,-0.014087,-0.047243,-0.046313,-0.011373,0.010814
3,1,39,1,-0.041668,-0.005339,-0.003069,-0.022583,-0.040599,0.011407,0.027688,0.028993,0.011937,0.037679
4,1,40,1,-0.031345,0.033751,-0.030880,-0.028066,0.028547,0.040212,0.015020,0.049602,-0.043647,0.025991


In [11]:
df['product_category'].max()

18

In [12]:
x_tmp = np.array(df['product_category'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(20, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['product_category_0', 'product_category_1', 
                                                   'product_category_2', 'product_category_3', 'product_category_4'])
X = pd.concat([X, X_tmp], axis=1)
X.head()

(30082800, 1)
300828/300828 [==============================] - 2s 8us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4
0,1,20,1,-0.020450,0.026225,-0.028412,-0.023482,-0.020356,-0.034429,-0.005427,0.026714,0.036977,0.003526,-0.013236,-0.042204,-0.009491,-0.005821,-0.010926
1,1,20,1,0.042897,0.042010,-0.013918,0.011240,-0.030617,-0.042416,-0.020023,0.015293,-0.038636,0.013217,-0.034503,0.042449,0.004390,0.020911,-0.025902
2,1,20,1,0.023067,0.027716,-0.028125,0.003288,-0.022204,-0.014087,-0.047243,-0.046313,-0.011373,0.010814,-0.013236,-0.042204,-0.009491,-0.005821,-0.010926
3,1,39,1,-0.041668,-0.005339,-0.003069,-0.022583,-0.040599,0.011407,0.027688,0.028993,0.011937,0.037679,-0.013236,-0.042204,-0.009491,-0.005821,-0.010926
4,1,40,1,-0.031345,0.033751,-0.030880,-0.028066,0.028547,0.040212,0.015020,0.049602,-0.043647,0.025991,-0.034503,0.042449,0.004390,0.020911,-0.025902


In [13]:
df['advertiser_id'].max()

62965

In [14]:
x_tmp = np.array(df['advertiser_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(62966, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['advertiser_id_0', 'advertiser_id_1', 
                                                   'advertiser_id_2', 'advertiser_id_3', 'advertiser_id_4'])
X = pd.concat([X, X_tmp], axis=1)
X.head()

300828/300828 [==============================] - 2s 7us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,1,20,1,-0.020450,0.026225,-0.028412,-0.023482,-0.020356,-0.034429,-0.005427,...,-0.013236,-0.042204,-0.009491,-0.005821,-0.010926,0.034189,0.045890,-0.033151,-0.031543,0.009927
1,1,20,1,0.042897,0.042010,-0.013918,0.011240,-0.030617,-0.042416,-0.020023,...,-0.034503,0.042449,0.004390,0.020911,-0.025902,-0.017913,0.037761,-0.017163,0.035041,-0.042916
2,1,20,1,0.023067,0.027716,-0.028125,0.003288,-0.022204,-0.014087,-0.047243,...,-0.013236,-0.042204,-0.009491,-0.005821,-0.010926,-0.019707,-0.029844,0.017956,-0.045425,-0.014875
3,1,39,1,-0.041668,-0.005339,-0.003069,-0.022583,-0.040599,0.011407,0.027688,...,-0.013236,-0.042204,-0.009491,-0.005821,-0.010926,-0.039117,-0.004306,-0.029039,-0.014768,-0.029305
4,1,40,1,-0.031345,0.033751,-0.030880,-0.028066,0.028547,0.040212,0.015020,...,-0.034503,0.042449,0.004390,0.020911,-0.025902,-0.039510,0.049428,-0.028880,0.046101,-0.002212


In [15]:
def check_matrix(x):
    if x.shape[0] >= 100:
        return x.iloc[:100, :]
    else:
        tmp1 = np.zeros((100 - x.shape[0], x.shape[1]))
        tmp1 = pd.DataFrame(tmp1)
        tmp1.columns = x.columns
        return pd.concat([x, tmp1], axis=0)
X = X.groupby('user_id').apply(check_matrix)
X = X.reset_index(drop=True)
X = X.drop('user_id', axis = 1)
X

,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,20.0,1.0,-0.020450,0.026225,-0.028412,-0.023482,-0.020356,-0.034429,-0.005427,0.026714,...,-0.013236,-0.042204,-0.009491,-0.005821,-0.010926,0.034189,0.045890,-0.033151,-0.031543,0.009927
1,20.0,1.0,0.042897,0.042010,-0.013918,0.011240,-0.030617,-0.042416,-0.020023,0.015293,...,-0.034503,0.042449,0.004390,0.020911,-0.025902,-0.017913,0.037761,-0.017163,0.035041,-0.042916
2,20.0,1.0,0.023067,0.027716,-0.028125,0.003288,-0.022204,-0.014087,-0.047243,-0.046313,...,-0.013236,-0.042204,-0.009491,-0.005821,-0.010926,-0.019707,-0.029844,0.017956,-0.045425,-0.014875
3,39.0,1.0,-0.041668,-0.005339,-0.003069,-0.022583,-0.040599,0.011407,0.027688,0.028993,...,-0.013236,-0.042204,-0.009491,-0.005821,-0.010926,-0.039117,-0.004306,-0.029039,-0.014768,-0.029305
4,40.0,1.0,-0.031345,0.033751,-0.030880,-0.028066,0.028547,0.040212,0.015020,0.049602,...,-0.034503,0.042449,0.004390,0.020911,-0.025902,-0.039510,0.049428,-0.028880,0.046101,-0.002212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89999995,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
89999996,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
89999997,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
89999998,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
# X.to_csv('data/train_preliminary/lstm_5.csv')

In [18]:
X = preprocessing.MinMaxScaler().fit_transform(X)


In [19]:
X = np.array(X).reshape((900000, 100, 22))

In [22]:
from keras.layers.core import Masking
X_train = X[:10000, :, :]
Y2_train = df1.iloc[:10000, 2:3]
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(100, 22)))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='MSE', optimizer='adam')
model.fit(X_train, Y2_train, nb_epoch=60)

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 1/100
10000/10000 [==============================] - 24s 2ms/step - loss: 0.2455
Epoch 2/100
10000/10000 [==============================] - 23s 2ms/step - loss: 0.2226
Epoch 3/100
10000/10000 [==============================] - 24s 2ms/step - loss: 0.2223
Epoch 4/100
10000/10000 [==============================] - 24s 2ms/step - loss: 0.2222
Epoch 5/100
10000/10000 [==============================] - 23s 2ms/step - loss: 0.2224
Epoch 6/100
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2218
Epoch 7/100
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2221
Epoch 8/100
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2223
Epoch 9/100
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2220
Epoch 10/100
10000/10000 [==============================] - 21s 2ms/step - loss: 0.2215
Epoch 11/100
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2215
Epoch 12/100
10000/10000 [===============

10000/10000 [==============================] - 22s 2ms/step - loss: 0.2125
Epoch 95/100
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2127
Epoch 96/100
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2133
Epoch 97/100
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2127
Epoch 98/100
10000/10000 [==============================] - 21s 2ms/step - loss: 0.2128
Epoch 99/100
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2131
Epoch 100/100
10000/10000 [==============================] - 21s 2ms/step - loss: 0.2127


In [26]:
Y2_train.shape

(10000, 1)

In [28]:
Y = model.predict(X_train)
Y = np.round(Y)
ans = 0
for i in range(10000):
    if Y[i] == Y2_train.iloc[i, 0]:
        ans += 1
print('accurency:', ans/10000)

accurency: 0.6842
